In [35]:
########################
#What I could do.
########################
#So far, in this notebook, I have implemented the item based, SVD and ALS algorithms.
#For each of these models, the RMSE, MSE and MAE have for test set has been calculated as evaluation metric
#I have also printed the estimated ratings for items on the test set for each algorithm.
#I have also added code for showing the estimated rating for a specific product for a specific user.
#All the above can be used to compare the three models.
########################

########################
#What I couldn't do.
########################
#There is a way within the surprise package to create an "anti_testset".
#This anti_testset basically is generated from the whole dataset.
#It takes those (user,item) pairs for which ratings are not available.
#Thus, fitting the model and making prediction on this anti_testset would be the ultimate goal
#since it would estimate how a user would rate a product that he hadn't rated before.
#However, when I try to create this anti_testset, it takes way too long, for this small dataset.
#Thus, it didn't seem feasible to me. Howewer, I left the code commented out. See if you can do anything.
########################

########################
#Miscellaneous
########################
#The basic approach would be to first run k-fold CV of Grid Search CV on multiple models and choose the best one
#based on the evaluation metrics (preferably RMSE).
#Then, for the selected "best" model, the predictions could be made (going through the anti_testset).
#But, for our project, since getting the best model is not the ultimate goal, rather a comparison among them will suffice,
#I have commented out the CV portions. In my opinion, we can discard those.
#I need to play around with the ALS for both Surprise and Pyspark to see how the execution time compares between them.
########################

#-------------------------------------------------Muhtasim

# 1. Data Preprocessing

## 1.1 Import Libraries

In [1]:
import time
import random
import numpy as np 
import pandas as pd 
from surprise import accuracy, Dataset, Reader, KNNWithMeans, SVD, BaselineOnly, Trainset
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from collections import defaultdict

## 1.2 Load Data

In [51]:
random.seed(123)
fname = "Software.csv"
mydata=pd.read_csv(f'./ratings_data/{fname}' ,names = ['item','user','rating','timestamp'])

del mydata['timestamp']
#reindex the column positions to be suitable for surprise
columns_titles = ['user','item','rating']
mydata=mydata.reindex(columns=columns_titles)
mydata.head()

,user,item,rating
0,A240ORQ2LF9LUI,0077613252,4.0
1,A1YCCU0YRLS0FE,0077613252,4.0
2,A1BJHRQDYVAY2J,0077613252,1.0
3,APRDVZ6QBIQXT,0077613252,3.0
4,A2JZTTBSLS1QXV,0077775473,5.0


## 1.3 Prepare data and check the number of users and items

In [52]:
# create a reader object to parse the dataset in the format appropriate for Surprise
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5))
# load the dataframe with reader as Surprise dataset object
data = Dataset.load_from_df(mydata, reader)
#with full dataset as training set
trainsetfull = data.build_full_trainset()
#check the unique items, users and ratings
print('Total number of users: ', trainsetfull.n_users,)
print('Total number of items: ', trainsetfull.n_items,)
print('Total number of ratings: ', trainsetfull.n_ratings,)
#print(mydata.user.nunique())
#print(mydata.item.nunique())
#print(mydata.rating.nunique())

Total number of users:  375147
Total number of items:  21663
Total number of ratings:  459436


# 2. Apply Collaborative Filtering using Sequential Algorithm

# 2.1 Memory based approach (item-based)

### Use Centered KNN to calculate similarity and check RMSE, MSE, MAE scores for test set

In [4]:
##############################################################
#memory based approach
##############################################################

#################
#item based
#################

#to use item-based cosine similarity
sim_options_item = {
    "name": "cosine",
    "user_based": False,  #compute similarities between items
}
#choose centered KNN as the algorithm for finding similar items
algo_item = KNNWithMeans(sim_options=sim_options_item)
#build the training set
trainset, testset = train_test_split(data, test_size=.20)
print('Number of users in trainset: ', trainset.n_users,)
print('Number of items in trainset: ', trainset.n_items,)
print('Number of ratings in trainset: ', trainset.n_ratings,)
#train the algorithm on the trainset, and predict ratings for the testset
algo_item.fit(trainset)
predictions_item = algo_item.test(testset)
#then compute RMSE, MSE and MAE
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_item)
accuracy.mse(predictions_item)
accuracy.mae(predictions_item)

Number of users in trainset:  308891
Number of items in trainset:  20089
Number of ratings in trainset:  367548
Computing the cosine similarity matrix...
Done computing similarity matrix.
The testing accuracy in RMSE is: 
RMSE: 1.5789
MSE: 2.4928
MAE:  1.3509


1.350935626340042

In [24]:
predictions_item

[Prediction(uid='A3DAD7S7DB1RXD', iid='B005EJ2ICU', r_ui=2.0, est=3.5686141674012646, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='A1J0QC8IRC67MC', iid='B015724OVG', r_ui=1.0, est=3.9481818181818182, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A115OIP4AYNQVF', iid='B00009EFYO', r_ui=4.0, est=3.1809523809523808, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='A3F8WHBAMXTL8W', iid='B000W3QKQS', r_ui=1.0, est=1.5787859643739974, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='AUY7W0K2SP9DP', iid='B009HBCU9W', r_ui=5.0, est=4.11839530332681, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A2BTL8M2I8DM0O', iid='B00604I9RG', r_ui=2.0, est=1, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='A114PKGTLBE6VI', iid='B00H9A60O4', r_ui=4.0, est=3.5686141674012646, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Predict

In [8]:
#for converting between raw (as strings, in the original data) 
#and inner IDs (integers, as converted by the surprise package itself)
#trainset_iids = list(trainset.all_items())
#iid_converter = lambda x: trainset.to_raw_iid(x)
#trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [9]:
#for cross validation, not necessary for our project
#results = cross_validate(
#    algo = algo_item, data = data, measures=['RMSE','MSE','MAE'], 
#    cv=5, return_train_measures=True
#    )
#print(results['test_rmse'].mean())
#print(results['test_mse'].mean())
#print(results['test_mae'].mean())

In [10]:
#takes long
#anti_testset = trainsetfull.build_anti_testset()

In [11]:
#takes way too long!
#predictions_antitest = algo_item.test(anti_testset)

### Check the estimated ratings on the testset

In [5]:
#estimated ratings on the testset
pred_df = pd.DataFrame(predictions_item)
pred_df[['uid', 'iid', 'est']]

,uid,iid,est
0,A3DAD7S7DB1RXD,B005EJ2ICU,3.568614
1,A1J0QC8IRC67MC,B015724OVG,3.948182
2,A115OIP4AYNQVF,B00009EFYO,3.180952
3,A3F8WHBAMXTL8W,B000W3QKQS,1.578786
4,AUY7W0K2SP9DP,B009HBCU9W,4.118395
...,...,...,...
91883,A2U44UJ2L3WUVD,B0153V62P2,3.568614
91884,A1DZT0GIVEDITP,B015724OVG,3.568614
91885,A1O3DM205FPZ9J,B00UB76290,3.568614
91886,AR3CQY145KF0M,B01326J5MC,3.568614


### Check the top recommendations for each user

In [6]:
#see the predictions on the test dataset
pred_df = pd.DataFrame(predictions_item)
del pred_df['details']
#finding recommendation for a user
pred_df[(pred_df['est']>0.5)&(pred_df['uid']=='APV13CM0919JD')]

#getting top 10 recommendations for each user
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions_item, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A3DAD7S7DB1RXD ['B005EJ2ICU']
A1J0QC8IRC67MC ['B015724OVG']
A115OIP4AYNQVF ['B00009EFYO']
A3F8WHBAMXTL8W ['B000W3QKQS', 'B000W3T6S2']
AUY7W0K2SP9DP ['B00FFINOWS', 'B009HBCU9W']
A2BTL8M2I8DM0O ['B00FYH912C', 'B00604I9RG']
A114PKGTLBE6VI ['B00H9A60O4']
A33R76NVDI9JFS ['B00UB76290']
A2CU0BTP0F1ODV ['B001AMLRU4']
A3QGWEB7GQLQS8 ['B00E6LJBUO']
A3HBD520RIUKEL ['B015JTT5L6']
A2DZ52ZINEVZ09 ['B00IT6WQDQ', 'B000Q1OTTG', 'B000PS6QY6']
A1ETFLRJSOYYR5 ['B0009MF770']
A1CQ0YHDT5QB5B ['B0012JK6DY']
A3OINUA7Z2W3FT ['B0095C08YM']
A6PNGQUHWLLQJ ['B00E6LJ2SA']
A2LHL3RYUYLCN8 ['B000VLZCEW', 'B000BQM01A']
A35H05NWQJ1O6I ['B00ND0E7BW']
A274P9J8BWF9F5 ['B00LHTRE1G']
A2JISJ1UHDAMKX ['B00C30U4HO']
AMRF6X8WH4150 ['B00FYH912C']
A2VLDYO8VK9G7K ['B007SA2QCA']
AUEYQZ8BZ9Q3A ['B002N5M57Q']
A14I9PNQCB9BHV ['B006264YCG']
A24AT8HZ2UE8O0 ['B00HV9IM58']
A2UQV92ZERS7GL ['B001SJHHZ8']
A35RNOOWR6F4YF ['B00ENFYM6Q']
A1AG6B659X54F3 ['B002U0L1BU']
A2CLCT73B5JV5F ['B0002ZI8SO', 'B00UB76290']
A39Z2M0O2K3XY2 ['B000TFBZEQ']
A1JBP0

A5GDQWP17H2QR ['B00MU0LDWW']
A1WNGH2BO1HKW7 ['B000V5PN06', 'B000VLZCEW']
AE1ZIBDX83MSE ['B00EOI2SR2']
A1U0FH83MJJ60R ['B00NG7JM62']
A74MSPY16IOW5 ['B000EIGCW8']
AFVT8Q1RRDXS0 ['B0039L6JIY']
A2R3KJI1NXTR7J ['B000IRWOKY']
AX1T63I2WHMNO ['B00EZPXYP4']
A101ZN5A94X3LD ['B00192D1HM']
AR5OQQA29W7FQ ['B00002SUW6']
A12433G3SCEG7U ['B015610XWW']
A11DQFGC3TT6U2 ['B00ABAW2UA']
A2OPB6J1BMHX9M ['B0013O98SW']
A2BFBYJW8ALHH2 ['B00CTTEKJW']
A13HSPC7PEQHWY ['B00002SA2H']
A23971D63L3JVG ['B007IWCXHG', 'B0039XZ3T8']
A25NDZRBFBSF7Q ['B00CTTEKJW']
A3L6ODO4IEWB49 ['B0000DBYKM', 'B0000DBYKP']
A2DJM0TM5K4JP5 ['B0002BQQTQ']
AY7Z0W9EKBCBB ['B017W85QUQ', 'B015IHWAZW', 'B015OY6WKS', 'B00OSCLQ1S']
AST490Y5U2L98 ['B00B1TGUMG']
A3HQ0VW060BVXG ['B009ENDZ4C']
A2Q3WE1KP2WHYL ['B004A7Y0UK']
AKE30C41GDAAS ['B001U3PYLQ']
A1EMEGNUOLAOSW ['B009KRW8JK']
AIBUJGS7P6SIY ['B00005LA8Y', 'B0009XB15G']
A3191P96NBU492 ['B00UB76290']
A2P4ENNHQ18T8Z ['B001FH8JDM']
A23S83UTHA9AMA ['B0000CE220']
A1U2SFIRC2MGML ['B001GL6QHS']
A1Z7GZQANXPY

A3MB8YL9HGSNEK ['B00FFIO0NA']
A290AWBCGK0CZO ['B0010YKYYM', 'B0010YKYYM']
AHI8SJJK7JUA0 ['B00H9A60O4']
A2IBZW41Y9U7B5 ['B001GL6QHS']
A3UFTCH49E5YDQ ['B00MYXTCGY']
A2ACLBGVEJXC1E ['B01F7RJHIQ', 'B006GOFW3E']
A347XZ21SVKU1V ['B00NG7K2RA']
A1HO70BR4HDVJW ['B011XO5KVK']
ARFIC8QLPTBNQ ['B00NG7JOWY']
ASG2Q4SFWSWK5 ['B00HRQB28Y']
AAH5YAER1ZBKQ ['B00NG7JVSQ']
A2T1MEI160223C ['B005S4YBVM']
A3P7KIATA96Q4B ['B00004WFVN']
A2295AQXEY9MK1 ['B00CTTEKJW']
A76J5OX3BIXZU ['B00A6TPHZS']
A115SCU64CSM6S ['B002VPE3FK']
A158NCNMLUVACW ['B0000C8Y0W']
A3ACNJ00ZYUO0A ['B00MHZ6Z64']
A13UWJ9AT5PFAG ['B003PDMNCC']
A39XNN6DZ40504 ['B004CRNACW']
AHAT651ISDSMS ['B008H3SW4I']
A748HWUXKYBBB ['B00LYYMXB0']
A3EY2N0PA203GT ['B005CELKLM']
A1FY5PRF448PNH ['B00BOV23M2']
AZGBENBPLO1A4 ['B00MHZ6Z64']
A1MRTZO4JT5P4H ['B00EZPXYP4']
A3J64I9NKSPG5H ['B002U2M5N6']
ANILI32QIKE9H ['B013XFGA6G']
A1RFITWHVLRBOC ['B00N3EZQFE']
A3EWSL99ZXWAGG ['B00JC5Y6YA']
A3UOM36NOE93SA ['B00D9ILK7W']
AQZ4PHIEEV5N5 ['B005N2CS8M', 'B010P91LYY']
A3GFJGRP

A17WW309XRZJJ7 ['B00LC9UU6C']
AVR6C4GIGDVZR ['B00MYXTCGY']
A2OZC62OBHACNI ['B00CTTEKJW']
A3K2WCXMT80YLO ['B00E5UHRAM']
A10Z4FG7WO508M ['B00R37PAIA']
AJIKS5GVAYQ7 ['B00H9A60O4']
A1866TSIL7JZS ['B008XAXAC4']
A2H6L05M8R4J2L ['B0064PFB9U']
A3GJT5AACJ35YN ['B00005AFI3']
A3LYWIKO0P3HX2 ['B00OTX1HBU']
A3D1EERAGTGH9T ['B00002S6E6']
AQUGN7LU02F8J ['B007A7JSMM']
A2W78KJ5N7KVT5 ['B001GLAVVU']
AIBXTHJFKM11X ['B00P31G9PQ', 'B00KQV8RNK']
A3058YBUMOLBQS ['B00MA5TPN6']
A1ILK0ZOKZTHJT ['B001327K8O']
A1OPS8Q346ZPRV ['B0094NY3R0']
A2F19R9V6RVPF5 ['B0007TITAW']
A2XBPJ4V623SW ['B015724RQI']
AMSK95UNMRARJ ['B001EJTO8A']
A1VJHR7MDVBKJA ['B005FIWT6U']
A3O1GPG85BQF1P ['B00LHTRE1G']
A1SENP9MS7MV61 ['B00ENFYLOO']
A2UY6DFWK1HQ8L ['B00NG7JVSQ']
A25J53EFQRT49N ['B0002XHBZ2']
A39YKDC6NL710F ['B00PG8FSYE']
AV4EFO9WZ2WFT ['B004Z7H1OW']
A43PSZQIV53Y ['B00H9A60O4']
A2L5QJ9ZSBGWHN ['B00MBYLU34']
A3QSRRIH4VIKRW ['B00EDSI7QO']
AZUKGJO1JOI18 ['B009SPL2K0', 'B0024M19PW']
A2AFFNK0XMCYN6 ['B005IIUFYU', 'B00RN05IGQ']
A1U5A50LBT

AFVG43AJO9ZHB ['B00MYXTCNC']
A1R2JY7WX99CAZ ['B00CTTEKJW']
A58XHOE1QEKTX ['B00H9A60O4']
AGRWEKOYCZINM ['B0079QJWPW', 'B0013FBFG4']
A33TEWRHTOZDU0 ['B00EDSI7QO']
A13G57IHYX7CAP ['B004230D62', 'B006GOFW3E']
A2Q8D5Q23LVWSW ['B00EZKNYXG']
A1I6VTR4T95540 ['B004KPKSRQ']
ASRDHBN0Z8GC5 ['B00UB76290', 'B00M9GTEPA']
A1B0YQZ7WYNX7D ['B00CTTEKJW']
A1EPLR6ECX0L0 ['B00RKZKFUI']
A3TZOD1AZK4D4L ['B00CLSTBIG']
A3LN8P1H96S5Y0 ['B00CTTEKJW', 'B00M76N6MO']
A198CCH4NNLI2E ['B00JZNHUFQ', 'B000AOBSTS', 'B00006FI0Z', 'B0000C8Y0W']
A3CAXSXM8GHSXI ['B0095CC1PQ']
AS80QORCQRPN5 ['B005CELHJC']
A2ULL8OFZQUIEP ['B00004NHKU']
A3SI8RUA9TAC5T ['B00CTTEKJW']
A2N1WT1EBTWTBV ['B00GOUBT6E']
AAIX10L57Y3CU ['B000KN44YA']
A2UQ9NHWXRMZPV ['B00B1TGUMG']
A2Y3CL5XJPK6WU ['B00E6LJ2SA']
A3B1ME89B42DAU ['B00UN8POM8']
A64PKBJKHSRTA ['B00G0DXA9Y']
A2GNAR1LF3Z8IA ['B016X8J198']
A1AO0BA5PGC9VF ['B0055A7RS8']
A1KXVU1LPDLT2 ['B000R2LXSY']
A34G2FV7UGT8WW ['B001B5MPNS']
A2RNCN62O1RZJA ['B000VI55YC']
A1FQU7VYKDUBO8 ['0615179088']
A1BUM0AQHWB

...

A1NW077L0L7TQ3 ['B008H3SW4I']
A1ZWTNM9AFLYG5 ['B00EOI2SR2']
ANDWC55MHW54M ['B00NG7JVSQ']
A39WHZ0BER1ODO ['B0123C60D8']
A22VTPKPQC0LW2 ['B00UB76290']
A19HIA7N6I8AL2 ['B0095C25Y8']
A1UBTEY3LEFXH6 ['B00NG7JVSQ']
AEGII5Q22R5CB ['B008RA5A00']
A2TQV43ZT6YQRC ['B000IK412K']
AZ6RFYXLQEQAC ['B00UB76290']
A1IDOHVQ07JEJB ['B016D48QDE']
A1KGVHTA4ZLD4U ['B002LLFAKG']
A3NGJCEULEZUK8 ['B008XAXAC4']
A3IYXMQAGOLVNN ['B00UB76290']
A2COSG3632HVNR ['B001B5FM7E']
A4GAU0R7X4GCF ['B00IP93PV4']
A33G6PNUH5NDLJ ['B00E6LJ1OA']
A3NY43GSR8X5G1 ['B000BS2V4Y']
A18BVWXG6FU0QP ['B0001H9L2K']
A193GXO7EAS64E ['B002VPE3FK']
A19P42XQFKBAML ['B0044DE8KM']
A1UPZ4G7YJWD4Q ['B00M76N6MO']
A5U5T6EWH90O0 ['B01FFVDY9M']
A3T29XBF6C2NIH ['B000HCZ8EO']
A2NYCHZ7MZ3OU3 ['B000BGQJXK']
AUA3UJV9QFKF4 ['B00MCLG0UY']
A3A6K45895MYN1 ['B003EYU5BI']
A3IYTQP1IBQGMV ['B00K7JFFQ8']
A13OPZZIZKDR4A ['B006SOMERY']
A2DK29JR1ANTFJ ['B0039L31JY']
A2A61Q2PPPVZJ3 ['B015724OVG']
A2BXOAJFZDLYFR ['B00PG8FOSY']
AOUA3YPECKL47 ['B003YJ5FMW']
A27R6HUFUMUT8H ['

A2W6RM6RWWT16P ['B00NG7JVSQ']
A3PU3URLYFXGP0 ['B01326J5MC']
A2R8QH3FB3J0N ['B00009APN9']
AVN7QSQE5NP5O ['B01326J80Q']
A2N8PI7AR1M0E8 ['B00BKWXNDI']
A1LMKQL051B5RH ['B00UB76290']
A2JH60SSY2YN31 ['B00G82I07Y']
A3TM3E7ZJTO9MD ['B004ISKTWE']
AW0WXQIIZWAZQ ['B00MUY6T6S']
AHLAW158Z6AJ6 ['B0166QDIN2']
A2QMYBWE0WE7S3 ['B004Q0T0LU']
A3CMV3HS9P1LGU ['B005FIWTHO']
A32072MUMFX83W ['B003O0NPB4']
A2YBJ24U6LFVYE ['B01F7RJHIQ']
A2LVR6M12THISG ['B000GHV6NC']
AR8006F9IJ54X ['B00EZKNY8G']
A28FN3QO2TDO99 ['B00FYH8URE']
A1UC3OZOZAT72L ['B015UAMV6A']
A28XETZMQNEUQK ['B0002T889Y']
A4U9UDZJSGXQM ['B00E6LIEFW']
A36IUQC3DQPFCX ['B0056CZBJC']
A3CK3JQD8YHW8W ['B008H3SW4I']
A2QWYRW3VTDKIY ['B01637RGG4']
ATOG3KYSGJWK1 ['B0051M6IPI']
A2KIHP208SGWDP ['B00NG7JVSQ']
A19KVHOG0DGM4P ['B000BJLN10']
A36STJM6X4RMEP ['B004DPKCO2']
A3KLPYPSIA649N ['B00MBYLU34']
A1A5IBHOGIRRVD ['B005FIWT74']
AW69HMGOTK0FE ['B00MYXTCIM']
AZRQV3XE9OO3V ['B00JC5Y6YA']
A1FEDN5ELIN843 ['B01617VQJ4']
A3VW29U8OMCCO6 ['B0013O54P8']
A2DZO2A2PLEF5S ['B0

A36M73TVN7434X ['B00MHZ6Z64']
A18D4V312564ZZ ['B000HCXKLC']
AM1WEJH7YIFJL ['B00MCLGAOU']
A3KN6D0B6LLKI1 ['B001B057U6']
A1XWA9B0T90HQ9 ['B00E6LJ2SA']
A2QSYC203E725Y ['B00E6OPDUS']
A12REZ679ID5IZ ['B001LQO4P4']
A3QCLAMOYB07AC ['B001F0RYY4']
AGMI7KX49XY6P ['B002PDPIF2']
A3BA9R572RIHWJ ['B000069IL7']
AMW45SL3UTD9R ['B0153V65J0']
A1D5WJ8QDWE68Y ['B001P3O764']
AIJ0YXAS7IXGS ['B00E6LJ1OA']
AZQHDTHONKNL5 ['B006GOFW3E']
A1LIZAJXVD1078 ['B000WCQCE4']
A199IFYP1IQ168 ['B00NG7JYYM']
A2EFYJ5PGMRRZY ['B000X86ZAS']
A2TY4YT8BE9MGZ ['B0055XNWJI']
AFDESJ4ZD5BSP ['B01617VO2S']
A1FNT1LFIJCAJA ['B00OVAGX38']
AESUOB7CGUPGE ['B01326J5MC']
A7S9KG6EM7W48 ['B0144NYEY6']
A1XRPCQ55105JM ['B00006K11Q']
A1JR860T787IFO ['B01CUQ17SU']
A16CN6JJJKL21I ['B00CTTEKJW']
A2LHCDF0EMI2M3 ['B00EZPXYP4']
A3V4TAYRSDIPHK ['B005CSF1JK']
A21GBI2AUHUAPN ['B000JX3OD8']
AHP1FIFQ3G8RW ['B00CTTEKJW']
A3MW1BPB7U9T3I ['B00PG8FOSY']
A2I967S98I1KM8 ['B011XO54NY']
AK7M5SF5GQDXN ['B00F8K9L66']
A33XEF24GQCVJH ['B000JD51ZW']
A3K9ZB2PCF2HG4 ['B00

A1M4QTDB413WXF ['B000XYUSMI']
A2GL3WLEN1USDG ['B006VRWPVS']
AW5Z996V76BNR ['B00P04JVCO']
A3AQD2A35IIC25 ['B0039L31JY']
AGLM8UY2VS7OH ['B00FFINUJK']
AZE7060DOOW2T ['B015724RQI']
A1A7EM3E559DB3 ['B0006LCHUE']
AG1F9OXDNHUR9 ['B00078UK2I']
ACKP19WG077GJ ['B0000BZ54N']
A1R9XZMFA7KA2S ['B002WJI3O2']
A295TA5TBS5R30 ['B0056CZC2S']
A2VTCL4HZ8CMID ['B00YARNNH6']
AY6E75WYI0L94 ['B00P6U8BA0']
AHOGXAQDNMSK1 ['B00004WJK9']
AE1Y5OQ3Z6UU3 ['B000EXU990']
A9KQH2Y6F0BVR ['B006GOFW3E']
A3O3AXUVZP7TX3 ['B0002UCSKI']
A21JA2VQ7QDNF2 ['B00H9A60O4']
A1F2E73XEGUZVO ['B01326J80Q']
A24AL8FVSWYFHU ['B009SPK1DY']
A2GGDR03UVJJUG ['B00ZSI7Y3U']
AN5X5H6KVKPPS ['B003PDMNCC']
A370E2W52D7JDE ['B00919RKEK']
A1DSGNHMAGNLTZ ['B000B8K7SG']
A1JVZ041BJ2ENB ['B004B6ATJQ']
A28LDZ6TU2MCH3 ['B0002KZ4V8']
AT58K84QJF7AO ['B010P91LXA']
AAZX7RUMW85BJ ['B00B766VZE']
A2JZTZN7YBP7MR ['B00M76N6MO']
A2KJNN47H5BM1G ['B00000K4CS']
AS3BZT2YKWHGU ['B01617VQJ4']
AXNZUVVY0QBDP ['B00NG7JVSQ']
A37DMSVUUL5DE6 ['B00BBJR0SO']
A3ITD2GNEMBXO3 ['B00EZQY

A33CTACW6C016F ['B00E6LJAGO']
A2CVV17BO8LM52 ['B000OV5L40']
A1UO23BJGYALSD ['B005GW7YNI']
A1N4EEQ1Q7RTJG ['B012P53OQO']
A3GTEZTUB7WFUS ['B00E6LJ2SA']
A3M8K1GEA7I0YQ ['B0000BZM38']
A53NKYW8QOS2L ['B003DO96FU']
A19XMHRB3G4DIR ['B000TMHZX4']
A2UKDTNN0IUOLF ['B01637RISK']
A2ABEE13W4XV8P ['B00NG7JVSQ']
A37PVOPGANWODX ['B01F7RJHIQ']
A3URXASDDH6S5Z ['B004F0CUPE']
A1O9XOK8BK0L74 ['B0010G9U9U']
A2179F5C4EW3MM ['B009HBDMAS']
AZQTI0ZTH66JC ['B008S0IV9G']
AN6PWAVKI6SEA ['B00MYXTCGY']
A343FRKA00U39D ['B000HCZ8EY']
A3RATUB6FPDGMB ['B000HCXKJY']
A199TGZFZF3IQ ['B00G3OILUI']
A384JHCW3EM4L6 ['B00UB76290']
A24DHBUK0PBYO5 ['B00005B6U7']
A36PWO2T4N0U3E ['B003GAMZD6']
A2OX25AU3M3ZJ4 ['B01637RJFM']
A33E31F9K24C7C ['B0095CATEG']
A2BYCP9L6GNX1V ['B0002UCJV6']
A3AGN22L0YLKXO ['B008RA5A00']
A3P8TYZJYGD6NB ['B000U83YKA']
A39G8L4KQF4Y3Z ['B00CTTEKJW']
A3RKUSMZCKX1B1 ['B00NLJ0XPO']
A2R1DY9UBTKWTD ['B00B1TGUMG']
AXO9OJQNB84OP ['B000VLZCEW']
A2Y9JCXD05H3PU ['B001CB3C4W']
AZITNW7MPAPCW ['B008H3SW4I']
A31PQJPG0L589T [

### Look at the estimate of how a specific user would rate a specific item

In [7]:
#predict rating for a single user
algo_item.predict(uid = 'ACRIUHXH5F6V8', iid = 'B001GXRQW0')

Prediction(uid='ACRIUHXH5F6V8', iid='B001GXRQW0', r_ui=None, est=3.5686141674012646, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [8]:
#################
#user based (doesn't work for this dataset)
#################

#to use user-based cosine similarity
sim_options_user = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
#choose centered KNN as the algorithm for finding similar suers
#algo_user = KNNWithMeans(sim_options=sim_options_user)
# Train the algorithm on the trainset, and predict ratings for the testset
#algo_user.fit(trainset)
#predictions_user = algo_user.test(testset)
# Then compute RMSE
#accuracy.rmse(predictions_user)

## 2.2 Model Based Approach

### Singular Value Decomposition (SVD)

In [9]:
#######################################################
#model based approach
#######################################################

###################
#SVD
###################
print('Using SVD')
#select the algorithm
algo_svd = SVD()
#fit the training set into the algorithm
algo_svd.fit(trainset)
predictions_svd = algo_svd.test(testset)
#then compute RMSE, MSE and MAE metrics of the test set
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_svd)
accuracy.mse(predictions_svd)
accuracy.mae(predictions_svd)

Using SVD
The testing accuracy in RMSE is: 
RMSE: 1.4939
MSE: 2.2318
MAE:  1.2733


1.2733114017166736

In [10]:
pred_df = pd.DataFrame(predictions_svd)
pred_df[['uid', 'iid', 'est']]

,uid,iid,est
0,A3DAD7S7DB1RXD,B005EJ2ICU,2.313180
1,A1J0QC8IRC67MC,B015724OVG,4.327440
2,A115OIP4AYNQVF,B00009EFYO,3.791069
3,A3F8WHBAMXTL8W,B000W3QKQS,2.139931
4,AUY7W0K2SP9DP,B009HBCU9W,4.541804
...,...,...,...
91883,A2U44UJ2L3WUVD,B0153V62P2,3.611871
91884,A1DZT0GIVEDITP,B015724OVG,3.934624
91885,A1O3DM205FPZ9J,B00UB76290,4.129259
91886,AR3CQY145KF0M,B01326J5MC,2.867259


### Alternating Least Squares (ALS) Sequential

In [57]:
###################
#ALS
###################

print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo_als = BaselineOnly(bsl_options=bsl_options)
#fit the training set into the algorithm
algo_als.fit(trainset)
predictions_als = algo_als.test(testset)
#then compute RMSE, MSE and MAE metrics of the test set
print("The testing accuracy in RMSE is: ")
accuracy.rmse(predictions_als)
accuracy.mse(predictions_als)
accuracy.mae(predictions_als)

Using ALS
Estimating biases using als...
The testing accuracy in RMSE is: 
RMSE: 1.5032
MSE: 2.2597
MAE:  1.2869


1.2869377831118998

In [58]:
fname = 'Software.csv'
meta_df = pd.read_csv(f'./meta_data/meta_{fname}').rename(columns={'asin':'item'})

In [62]:
#recommend top 6 items based on ALS
top_n = get_top_n(predictions_als, n=6)

# Print the recommended items for each user
def get_recommendations(userid, n=6):
    return top_n[userid][:6]

def print_recommendations(userid, n=6):
    msg = f'Top recommended items for user {userid}:'
    print(msg)
    print('-'*len(msg))
    for i, tup in enumerate(get_recommendations(userid, n), 1):
        row = meta_df.loc[meta_df.item == tup[0]].iloc[0]
        item = row['title']
        print(str(i) + '.' + ' ' + item)
        print(' '*5 + 'https://www.amazon.com/dp/{}'.format(row['item']) + '\n')

print_recommendations('A22S7D0LP8GRDH')

Top recommended items for user A22S7D0LP8GRDH:
----------------------------------------------
1. The Expositor's Bible Commentary 5.0 for Windows : The Complete Award-Winning 12-Volume Commentary
     https://www.amazon.com/dp/031024837X

2. Norton Antivirus 2009 [OLD VERSION]
     https://www.amazon.com/dp/B001E7788K

3. WinZip 18 Pro (Old Version)
     https://www.amazon.com/dp/B00GDF83SM

4. Microsoft Office 365 Home | 1-year subscription, 5 users, PC/Mac Key Card
     https://www.amazon.com/dp/0763855553

5. Rosetta Stone Version 3: German Level 1, 2 &amp; 3 Set with Audio Companion
     https://www.amazon.com/dp/B001AFFRM0

6. CyberLink PowerDirector 10 Ultra [Old Version]
     https://www.amazon.com/dp/B005S7ZLU4

